# Cross-validation

Cross-validation is a kind of resampling approach that can be used to estimate the test error associated with a given statistical learning method in order to evaluate its performance, or to select the appropriate level of flexibility. The process of evaluating a model’s performance is known as _model assessment_, whereas the process of selecting the proper level of flexibility for a model is known as _model selection_.

Watch the 6-minute video below for a visual explanation of cross-validation:

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/fSytzGwwBVw?start=15" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining Cross Validation, by StatQuest](https://www.youtube.com/embed/fSytzGwwBVw?start=15)
```


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (
    train_test_split,
    LeaveOneOut,
    KFold,
    cross_val_score,
)
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline

## Load dataset

We will illustrate cross-validation on the `Auto` dataset. Run the following cell to load this dataset.

In [ ]:
auto_url = "https://github.com/pykale/transparentML/raw/main/data/Auto.csv"

auto_df = pd.read_csv(auto_url, na_values="?").dropna()
auto_df.info()

## Validation Set Approach

<!-- Using [Polynomial](http://scikit-learn.org/dev/modules/preprocessing.html#generating-polynomial-features) feature generation in scikit-learn -->
The validation set approach is a strategy to estimate the _test error_ associated with fitting a particular model on a set of observations. It involves randomly dividing the available set of observations into two parts, a _training set_ and a _validation set_ or _hold-out set_. The model is fit on the training set, and the fitted model is used to predict the responses for the unseen observations in the validation set. The resulting validation set error rate—typically assessed using MSE in the case of a quantitative response—provides an estimate of the test error rate.

Run the following to illustrate the validation set approach on the `Auto` dataset over different polynomial degrees.

In [ ]:
t_prop = 0.5
p_order = np.arange(1, 11)
r_state = np.arange(0, 10)

X, Y = np.meshgrid(p_order, r_state, indexing="ij")
Z = np.zeros((p_order.size, r_state.size))

regr = LinearRegression()

# Generate 10 random splits of the dataset
for (i, j), v in np.ndenumerate(Z):
    poly = PolynomialFeatures(int(X[i, j]))
    X_poly = poly.fit_transform(auto_df.horsepower.values.reshape(-1, 1))

    X_train, X_test, y_train, y_test = train_test_split(
        X_poly, auto_df.mpg.ravel(), test_size=t_prop, random_state=Y[i, j]
    )

    regr.fit(X_train, y_train)
    pred = regr.predict(X_test)
    Z[i, j] = mean_squared_error(y_test, pred)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Left plot (first split)
ax1.plot(X.T[0], Z.T[0], "-o")
ax1.set_title("Random split of the data set")

# Right plot (all splits)
ax2.plot(X, Z)
ax2.set_title("10 random splits of the data set")

for ax in fig.axes:
    ax.set_ylabel("Mean Squared Error")
    ax.set_ylim(15, 30)
    ax.set_xlabel("Degree of Polynomial")
    ax.set_xlim(0.5, 10.5)
    ax.set_xticks(range(2, 11, 2));

In [Linear regression](https://pykale.github.io/transparentML/02-linear-reg/extension-limitation.html#non-linear-relationships), we discovered a non-linear relationship between `mpg` and `horsepower`. Compared to using only a liner term, a model that using `horsepower` and `horsepower`$^2$ gives better results in predicts `mpg`. For the `Auto` dataset, we can randomly split the 392 observations into two sets, a training set containing 196 of the data points for model training, and a validation set containing the remaining 196 observations for evaluating by MSE. As shown in the left-hand panel of the figure above.

If we repeat the process of randomly splitting, we will get a somewhat different estimate for the test MSE. Ten different validation set MSE curves are shown in the right-hand panel of the figure above.
<!-- As an illustration, the right-hand panel of Figure 5.2 displays ten different validation set MSE curves from the Auto data set, produced using ten different random splits of the observations into training and validation sets.  -->
We can observe:
- model with a quadratic term has a dramatically smaller validation set MSE than the model with only a linear term
- ot much benefit in including cubic or higher-order polynomial terms in the model
- each of the ten curves results in a different test MSE estimate for each of the ten regression models considered 
- there is no consensus among the curves as to which model results in the smallest validation set MSE. 

Based on the variability among these curves, all that we can conclude with any confidence is that the linear fit is not adequate for this data. The validation set approach is conceptually simple and is easy to implement. But it has two potential drawbacks:

1. As is shown in the right-hand panel of Figure 5.2, the validation estimate of the test error rate can be highly variable, depending on precisely which observations are included in the training set and which observations are included in the validation set.
2. In the validation approach, only a subset of the observations—those that are included in the training set rather than in the validation set—are used to fit the model. Since statistical methods tend to perform worse when trained on fewer observations, this suggests that the validation set error rate may tend to overestimate the test error rate for the model fit on the entire data set.
   
In the coming subsections, we will present cross-validation, a refinement of the validation set approach that addresses these two issues.

## Leave-One-Out Cross-Validation

Leave-one-out cross-validation (LOOCV) is closely related to the validation set approach of Section 5.1.1, but it attempts to address that method’s drawbacks.

Like the validation set approach, LOOCV involves splitting the set of observations into two parts. However, instead of creating two subsets of comparable size, a single observation (x 1 , y 1 ) is used for the validation set, and the remaining observations {(x 2 , y 2 ), . . . , (x n , y n )} make up the training set. The statistical learning method is fit on the n − 1 training observations, and a prediction ŷ 1 is made for the excluded observation, using its value x 1 . Since (x 1 , y 1 ) was not used in the fitting process, MSE 1 = (y 1 − ŷ 1 ) 2 provides an approximately unbiased estimate for the test error. But even though MSE 1 is unbiased for the test error, it is a poor estimate because it is highly variable, since it is based upon a single observation
(x 1 , y 1 ).

We can repeat the procedure by selecting (x 2 , y 2 ) for the validation data, training the statistical learning procedure on the n − 1 observations {(x 1 , y 1 ), (x 3 , y 3 ), . . . , (x n , y n )}, and computing MSE 2 = (y 2 −ŷ 2 ) 2 . Repeating this approach n times produces n squared errors, MSE 1 , . . . , MSE n . The LOOCV estimate for the test MSE is the average of these n test error estimates:

$$\frac{1}{n}\sum_{i=1}^n MSE_i$$

A schematic of the LOOCV approach is illustrated in Figure 5.3.

LOOCV has a couple of major advantages over the validation set approach. First, it has far less bias. In LOOCV, we repeatedly fit the statistical learning method using training sets that contain n − 1 observations, almost as many as are in the entire data set. This is in contrast to the validation set approach, in which the training set is typically around half the size of the original data set. Consequently, the LOOCV approach tends not to overestimate the test error rate as much as the validation set approach does. Second, in contrast to the validation approach which will yield different results when applied repeatedly due to randomness in the training/validation set splits, performing LOOCV multiple times will always yield the same results: there is no randomness in the training/validation set splits.

We used LOOCV on the Auto data set in order to obtain an estimate of the test set MSE that results from fitting a linear regression model to predict mpg using polynomial functions of horsepower . The results are shown in the left-hand panel of Figure 5.4.

LOOCV has the potential to be expensive to implement, since the model has to be fit n times. This can be very time consuming if n is large, and if each individual model is slow to fit. With least squares linear or polynomial regression, an amazing shortcut makes the cost of LOOCV the same as that of a single model fit! The following formula holds:



where ŷ i is the ith fitted value from the original least squares fit, and h i is the leverage defined in (3.37) on page 99. 1 This is like the ordinary MSE, except the ith residual is divided by 1 − h i . The leverage lies between 1/n and 1, and reflects the amount that an observation influences its own fit.
Hence the residuals for high-leverage points are inflated in this formula by exactly the right amount for this equality to hold.
LOOCV is a very general method, and can be used with any kind of predictive modeling. For example we could use it with logistic regression or linear discriminant analysis, or any of the methods discussed in later chapters. The magic formula (5.2) does not hold in general, in which case the model has to be refit n times.


In [ ]:
p_order = np.arange(1, 11)
r_state = np.arange(0, 10)

# LeaveOneOut CV
regr = LinearRegression()
loo = LeaveOneOut()
loo.get_n_splits(auto_df)
scores = list()

for i in p_order:
    poly = PolynomialFeatures(i)
    X_poly = poly.fit_transform(auto_df.horsepower.values.reshape(-1, 1))
    score = cross_val_score(
        regr, X_poly, auto_df.mpg, cv=loo, scoring="neg_mean_squared_error"
    ).mean()
    scores.append(score)

In [ ]:
# k-fold CV
folds = 10
elements = len(auto_df.index)

X, Y = np.meshgrid(p_order, r_state, indexing="ij")
Z = np.zeros((p_order.size, r_state.size))

regr = LinearRegression()

for (i, j), v in np.ndenumerate(Z):
    poly = PolynomialFeatures(X[i, j])
    X_poly = poly.fit_transform(auto_df.horsepower.values.reshape(-1, 1))
    kf_10 = KFold(n_splits=folds, random_state=Y[i, j], shuffle=True)
    Z[i, j] = cross_val_score(
        regr, X_poly, auto_df.mpg, cv=kf_10, scoring="neg_mean_squared_error"
    ).mean()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Note: cross_val_score() method return negative values for the scores.
# https://github.com/scikit-learn/scikit-learn/issues/2439

# Left plot
ax1.plot(p_order, np.array(scores) * -1, "-o")
ax1.set_title("LOOCV")

# Right plot
ax2.plot(X, Z * -1)
ax2.set_title("10-fold CV")

for ax in fig.axes:
    ax.set_ylabel("Mean Squared Error")
    ax.set_ylim(15, 30)
    ax.set_xlabel("Degree of Polynomial")
    ax.set_xlim(0.5, 10.5)
    ax.set_xticks(range(2, 11, 2));

## k-Fold cross-validation

An alternative to LOOCV is k-fold CV. This approach involves randomly dividing the set of observations into k groups, or folds, of approximately equal size. The first fold is treated as a validation set, and the method is fit on the remaining k − 1 folds. The mean squared error, MSE 1 , is then computed on the observations in the held-out fold. This procedure is repeated k times; each time, a different group of observations is treated as a validation set. This process results in k estimates of the test error, MSE 1 , MSE 2 , . . . , MSE k . The k-fold CV estimate is computed by averaging these values, 

It is not hard to see that LOOCV is a special case of k-fold CV in which k is set to equal n. In practice, one typically performs k-fold CV using k = 5 or k = 10. What is the advantage of using k = 5 or k = 10 rather than k = n? The most obvious advantage is computational. LOOCV requires fitting the statistical learning method n times. This has the potential to be computationally expensive (except for linear models fit by least squares, in which case formula (5.2) can be used). But cross-validation is a very general approach that can be applied to almost any statistical learning method. Some statistical learning methods have computationally intensive fitting procedures, and so performing LOOCV may pose computational problems, especially if n is extremely large. In contrast, performing 10-fold CV requires fitting the learning procedure only ten times, which may be much more feasible. As we see in Section 5.1.4, there also can be other non-computational advantages to performing 5-fold or 10-fold CV, which involve the bias-variance trade-oﬀ.

The right-hand panel of Figure 5.4 displays nine different 10-fold CV estimates for the Auto data set, each resulting from a different random split of the observations into ten folds. As we can see from the figure, there is some variability in the CV estimates as a result of the variability in how the observations are divided into ten folds. But this variability is typically much lower than the variability in the test error estimates that results from the validation set approach (right-hand panel of Figure 5.2).

When we examine real data, we do not know the true test MSE, and so it is difficult to determine the accuracy of the cross-validation estimate. However, if we examine simulated data, then we can compute the true test MSE, and can thereby evaluate the accuracy of our cross-validation results. In Figure 5.6, we plot the cross-validation estimates and true test error rates that result from applying smoothing splines to the simulated data sets illustrated in Figures 2.9–2.11 of Chapter 2. The true test MSE is displayed in blue. The black dashed and orange solid lines respectively show the estimated LOOCV and 10-fold CV estimates. In all three plots, the two cross-validation estimates are very similar. In the right-hand panel of Figure 5.6, the true test MSE and the cross-validation curves are almost identical. In the center panel of Figure 5.6, the two sets of curves are similar at the lower degrees of flexibility, while the CV curves overestimate the test set MSE for higher degrees of flexibility. In the left-hand panel of Figure 5.6, the CV curves have the correct general shape, but they underestimate the
true test MSE.

When we perform cross-validation, our goal might be to determine how well a given statistical learning procedure can be expected to perform on independent data; in this case, the actual estimate of the test MSE is of interest. But at other times we are interested only in the location of the minimum point in the estimated test MSE curve. This is because we might be performing cross-validation on a number of statistical learning methods, or on a single method using different levels of flexibility, in order
to identify the method that results in the lowest test error. For this purpose, the location of the minimum point in the estimated test MSE curve is important, but the actual value of the estimated test MSE is not. We find in Figure 5.6 that despite the fact that they sometimes underestimate the true test MSE, all of the CV curves come close to identifying the correct level of flexibility—that is, the flexibility level corresponding to the smallest test MSE.

More [cross-validation strategies in scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html)